In [1]:
import cv2
import mediapipe as mp
import json
import os

# Mediapipe kurulumu
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Video klasörü ve çıkış klasörü
video_folder = "İşaretDiliVideo"
output_folder = "json_verileri"

os.makedirs(output_folder, exist_ok=True)  # Çıkış klasörünü oluştur

# Videoları işle
for video_file in os.listdir(video_folder):
    if not video_file.endswith(".mp4"):
        continue  # Sadece MP4 dosyaları işle
    
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)
    holistic = mp_holistic.Holistic()

    keypoints_data = []  # Hareketleri saklayacağımız liste

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        frame_data = {}

        if results.left_hand_landmarks:
            frame_data["left_hand"] = [[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]
        
        if results.right_hand_landmarks:
            frame_data["right_hand"] = [[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]

        keypoints_data.append(frame_data)

    cap.release()

    # JSON dosyasına kaydet
    json_path = os.path.join(output_folder, video_file.replace(".mp4", ".json"))
    with open(json_path, "w") as f:
        json.dump(keypoints_data, f)
    
    print(f"{video_file} işlendi ve {json_path} olarak kaydedildi.")

Ben.mp4 işlendi ve json_verileri\Ben.json olarak kaydedildi.
geliştiriyorum.mp4 işlendi ve json_verileri\geliştiriyorum.json olarak kaydedildi.
Merhaba.mp4 işlendi ve json_verileri\Merhaba.json olarak kaydedildi.
Proje.mp4 işlendi ve json_verileri\Proje.json olarak kaydedildi.
Öğrencisiyim.mp4 işlendi ve json_verileri\Öğrencisiyim.json olarak kaydedildi.
Üniversite.mp4 işlendi ve json_verileri\Üniversite.json olarak kaydedildi.
Şimdi.mp4 işlendi ve json_verileri\Şimdi.json olarak kaydedildi.


In [6]:
import open3d as o3d
import numpy as np
import time
import json
import os
from PIL import Image

# Klasör yolları
json_folder = "json_verileri"
animation_output_folder = "animasyonlar"

os.makedirs(animation_output_folder, exist_ok=True)

# JSON dosyalarını işle
for json_file in os.listdir(json_folder):
    if not json_file.endswith(".json"):
        continue

    json_path = os.path.join(json_folder, json_file)

    # JSON dosyasını aç ve veriyi yükle
    try:
        with open(json_path, "r") as f:
            keypoints = json.load(f)
    except json.JSONDecodeError:
        print(f"⚠ JSON dosyası okunamadı: {json_file}")
        continue

    if not keypoints:
        print(f"⚠ {json_file} dosyası boş, işlem atlandı.")
        continue

    # Open3D Robot Modeli
    torso = o3d.geometry.TriangleMesh.create_sphere(radius=0.15)
    shoulder = o3d.geometry.TriangleMesh.create_cylinder(radius=0.05, height=0.2)
    elbow = o3d.geometry.TriangleMesh.create_cylinder(radius=0.05, height=0.2)
    wrist = o3d.geometry.TriangleMesh.create_cylinder(radius=0.05, height=0.2)
    hand = o3d.geometry.TriangleMesh.create_sphere(radius=0.08)

    # Renkler
    torso.paint_uniform_color([0.2, 0.2, 0.8])
    shoulder.paint_uniform_color([0.8, 0.2, 0.2])
    elbow.paint_uniform_color([0.8, 0.2, 0.2])
    wrist.paint_uniform_color([0.8, 0.2, 0.2])
    hand.paint_uniform_color([0.2, 0.8, 0.2])

    # Başlangıç Pozisyonu
    shoulder.translate([0, 0, 0.2])
    elbow.translate([0, 0, 0.4])
    wrist.translate([0, 0, 0.6])
    hand.translate([0, 0, 0.8])

    # Open3D Görselleştirme Başlat
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=True)

    vis.add_geometry(torso)
    vis.add_geometry(shoulder)
    vis.add_geometry(elbow)
    vis.add_geometry(wrist)
    vis.add_geometry(hand)

    frame_images = []

    # Görselleştirme ve Kare Kaydı
    for frame_idx, frame_data in enumerate(keypoints):
        try:
            # Pozisyon Güncellemelerini Atla
            vis.update_geometry(shoulder)
            vis.update_geometry(elbow)
            vis.update_geometry(wrist)
            vis.update_geometry(hand)
            vis.poll_events()
            vis.update_renderer()

            # Kareyi kaydet
            frame_path = os.path.join(animation_output_folder, f"{json_file.replace('.json', '')}_{frame_idx}.png")
            vis.capture_screen_image(frame_path)
            time.sleep(0.2)

            if os.path.exists(frame_path):
                print(f"✅ Kare kaydedildi: {frame_path}")
                frame_images.append(frame_path)
            else:
                print(f"⚠ Kare kaydedilemedi: {frame_path}")

        except Exception as e:
            print(f"⚠ Kare işleme sırasında hata oluştu: {e}")
            continue

    vis.destroy_window()

    # Open3D Karelerini GIF'e Dönüştür
    gif_path = os.path.join(animation_output_folder, json_file.replace(".json", ".gif"))

    if frame_images:
        try:
            frames = [Image.open(frame) for frame in frame_images]
            frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=100, loop=0)

            # Geçici kareleri temizle
            for img in frame_images:
                os.remove(img)

            print(f"✅ Animasyon {gif_path} olarak kaydedildi.")
        except Exception as e:
            print(f"❌ GIF oluşturulurken hata oluştu: {e}")
    else:
        print(f"⚠ GIF oluşturulamadı, {json_file} için hiçbir kare kaydedilmedi.")

✅ Kare kaydedildi: animasyonlar\Ben_0.png
✅ Kare kaydedildi: animasyonlar\Ben_1.png
✅ Kare kaydedildi: animasyonlar\Ben_2.png
✅ Kare kaydedildi: animasyonlar\Ben_3.png
✅ Kare kaydedildi: animasyonlar\Ben_4.png
✅ Kare kaydedildi: animasyonlar\Ben_5.png
✅ Kare kaydedildi: animasyonlar\Ben_6.png
✅ Kare kaydedildi: animasyonlar\Ben_7.png
✅ Kare kaydedildi: animasyonlar\Ben_8.png
✅ Kare kaydedildi: animasyonlar\Ben_9.png
✅ Kare kaydedildi: animasyonlar\Ben_10.png
✅ Kare kaydedildi: animasyonlar\Ben_11.png
✅ Kare kaydedildi: animasyonlar\Ben_12.png
✅ Kare kaydedildi: animasyonlar\Ben_13.png
✅ Kare kaydedildi: animasyonlar\Ben_14.png
✅ Kare kaydedildi: animasyonlar\Ben_15.png
✅ Kare kaydedildi: animasyonlar\Ben_16.png
✅ Kare kaydedildi: animasyonlar\Ben_17.png
✅ Kare kaydedildi: animasyonlar\Ben_18.png
✅ Kare kaydedildi: animasyonlar\Ben_19.png
✅ Kare kaydedildi: animasyonlar\Ben_20.png
✅ Kare kaydedildi: animasyonlar\Ben_21.png
✅ Kare kaydedildi: animasyonlar\Ben_22.png
✅ Kare kaydedildi: an

In [1]:
import os
import json

def merge_json_files(json_folder, output_json):
    all_frames = []
    
    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, "r") as f:
                data = json.load(f)
                if "frames" in data:
                    all_frames.extend(data["frames"])
    
    merged_data = {"frames": all_frames}
    with open(output_json, "w") as out_file:
        json.dump(merged_data, out_file, indent=4)
    
    print(f"Merged JSON saved at {output_json}")

def main():
    json_folder = "jsonDosyası"  # JSON dosyalarının bulunduğu klasör
    output_json = "merged_frames.json"  # Çıktı dosyası
    merge_json_files(json_folder, output_json)

if __name__ == "__main__":
    main()

Merged JSON saved at merged_frames.json


In [15]:
import os
import json
import cv2
import speech_recognition as sr

def merge_json_files(json_folder, output_json):
    all_frames = {}
    
    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, "r") as f:
                data = json.load(f)
                word = filename.replace(".json", "").lower()  # JSON dosya adı kelime olarak alınır
                all_frames[word] = data.get("frames", [])
    
    with open(output_json, "w") as out_file:
        json.dump(all_frames, out_file, indent=4)
    
    print(f"Merged JSON saved at {output_json}")

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşmanızı bekliyorum...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        text = recognizer.recognize_google(audio, language="tr-TR")
        print("Algılanan metin:", text)
        return text.lower()
    except sr.UnknownValueError:
        print("Ses anlaşılamadı.")
    except sr.RequestError:
        print("Ses tanıma servisine ulaşılamadı.")
    return ""

def get_video_frames(json_file, sentence):
    with open(json_file, "r") as f:
        data = json.load(f)
    
    words = sentence.split()
    frame_list = []
    
    for word in words:
        if word in data:
            print(f"Kelime bulundu: {word}, {len(data[word])} frame var.")
            for frame in data[word]:
                img_path = frame.get("image_path").replace("\\", "/")
                if os.path.exists(img_path):
                    img = cv2.imread(img_path)
                    frame_list.append(img)
        else:
            print(f"Uyarı: '{word}' kelimesi için frame bulunamadı.")
    
    if not frame_list:
        print("No frames loaded. Check file paths and JSON structure.")
    
    return frame_list

def create_video(frames, output_video, fps=30):
    if not frames:
        print("No frames to create a video.")
        return
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"Video saved at {output_video}")

def main():
    json_folder = "jsonDosyası"  # JSON dosyalarının olduğu klasör
    output_json = "merged_frames2.json"  # Birleştirilmiş JSON
    merge_json_files(json_folder, output_json)
    
    sentence = recognize_speech()
    if sentence:
        video_frames = get_video_frames(output_json, sentence)
        create_video(video_frames, "output_video.mp4")

if __name__ == "__main__":
    main()

Merged JSON saved at merged_frames2.json
Konuşmanızı bekliyorum...
Algılanan metin: Merhaba ben
Kelime bulundu: merhaba, 141 frame var.
Kelime bulundu: ben, 72 frame var.
Video saved at output_video.mp4


In [6]:
import os
import json

def merge_json_files(json_folder, output_json):
    all_frames = []
    
    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, "r") as f:
                data = json.load(f)
                if "frames" in data:
                    for frame in data["frames"]:
                        if "image_path" in frame:
                            frame["image_path"] = frame["image_path"].replace("\\", "/")
                    all_frames.extend(data["frames"])
    
    merged_data = {"frames": all_frames}
    with open(output_json, "w") as out_file:
        json.dump(merged_data, out_file, indent=4)
    
    print(f"Merged JSON saved at {output_json}")

def main():
    json_folder = "jsonDosyası"  # JSON dosyalarının bulunduğu klasör
    output_json = "merged_frames2.json"  # Çıktı dosyası
    merge_json_files(json_folder, output_json)

if __name__ == "__main__":
    main()

Merged JSON saved at merged_frames2.json


In [23]:
import json
import cv2
import imageio
import os
import numpy as np
import speech_recognition as sr

def mikrofondan_metne_cevir():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşmaya başlayın...")
        recognizer.adjust_for_ambient_noise(source)
        ses_verisi = recognizer.listen(source)
    try:
        metin = recognizer.recognize_google(ses_verisi, language="tr-TR")
        print("Algılanan metin:", metin)
        kelimeler = metin.lower().split()
        return [kelime.strip(",.!?") for kelime in kelimeler]  # Noktalama işaretlerini temizle
    except sr.UnknownValueError:
        print("Ses anlaşılamadı.")
    except sr.RequestError:
        print("Servise ulaşılamadı.")
    return []

def json_dosyasindan_animasyon_olustur(json_dosyasi, kelimeler, cikis_gif):
    image_list = []
    
    if not os.path.exists(json_dosyasi):
        print("JSON dosyası bulunamadı!")
        return
    
    with open(json_dosyasi, "r", encoding="utf-8") as f:
        data = json.load(f)

    for kelime in kelimeler:
        if kelime in data:
            frames = data[kelime].get("frames", [])
            
            for frame in frames:
                image_path = frame.get("image_path", "")
                if os.path.exists(image_path):
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    image_list.append(np.array(img))
        else:
            print(f"Uyarı: '{kelime}' için JSON dosyasında bir kayıt bulunamadı.")

    if image_list:
        min_shape = min([img.shape for img in image_list], key=lambda x: x[0] * x[1])
        image_list = [cv2.resize(img, (min_shape[1], min_shape[0])) for img in image_list]
        imageio.mimsave(cikis_gif, image_list, fps=10)
        print(f"Animasyon kaydedildi: {cikis_gif}")
    else:
        print("Hiçbir geçerli görüntü bulunamadı.")

# Kullanım
json_dosyasi = "all_frames.json"  # Tek bir JSON dosyası
cikis_gif = "animation.gif"  # Çıkış animasyonu

kelimeler = mikrofondan_metne_cevir()
json_dosyasindan_animasyon_olustur(json_dosyasi, kelimeler, cikis_gif)


Konuşmaya başlayın...
Algılanan metin: Merhaba
Uyarı: 'merhaba' için JSON dosyasında bir kayıt bulunamadı.
Hiçbir geçerli görüntü bulunamadı.


In [22]:
import os
import json
import cv2
import speech_recognition as sr

def merge_json_files(json_folder, output_json):
    """ Tüm JSON dosyalarını birleştirir ve tek bir JSON dosyası oluşturur. """
    all_frames = {}
    
    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                word = filename.replace(".json", "").lower()  # JSON dosya adı kelime olarak alınır
                all_frames[word] = sorted(data.get("frames", []), key=lambda x: x["frame_id"])  # Frame'leri sıraya koy
    
    with open(output_json, "w", encoding="utf-8") as out_file:
        json.dump(all_frames, out_file, indent=4)
    
    print(f"Merged JSON saved at {output_json}")

def recognize_speech():
    """ Kullanıcının konuşmasını tanır ve metin olarak döndürür. """
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşmanızı bekliyorum...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        text = recognizer.recognize_google(audio, language="tr-TR")
        print("Algılanan metin:", text)
        return text.lower()
    except sr.UnknownValueError:
        print("Ses anlaşılamadı.")
    except sr.RequestError:
        print("Ses tanıma servisine ulaşılamadı.")
    return ""

def get_video_frames(json_file, sentence):
    """ Cümledeki kelimelere göre ilgili frame'leri JSON'dan alır. """
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    words = sentence.lower().split()  # Cümledeki kelimeleri küçük harfe çevirerek ayır
    frame_list = []

    for word in words:
        best_match = None
        for key in data.keys():
            if word in key:  # Kelime anahtar içinde geçiyorsa eşleşme kabul edilir
                best_match = key
                break  # İlk eşleşeni al, daha fazla arama

        if best_match:
            print(f"Kelime bulundu: {best_match}, {len(data[best_match])} frame var.")
            for frame in sorted(data[best_match], key=lambda x: x["frame_id"]):  # Frame'leri sıralı ekle
                img_path = os.path.join("frames", os.path.basename(frame.get("image_path")))  # Yol formatını düzelt
                img_path = img_path.replace("\\", "/")

                if os.path.exists(img_path):
                    img = cv2.imread(img_path)
                    frame_list.append(img)
                else:
                    print(f"Hata: {img_path} bulunamadı!")
        else:
            print(f"Uyarı: '{word}' kelimesi için frame bulunamadı.")

    if not frame_list:
        print("No frames loaded. Check file paths and JSON structure.")

    return frame_list

def create_video(frames, output_video, fps=10):
    """ Frame listesini birleştirerek video oluşturur ve kaydeder. """
    if not frames:
        print("No frames to create a video.")
        return
    
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    for frame in frames:
        out.write(frame)
    
    out.release()
    print(f"Video başarıyla {output_video} olarak kaydedildi.")

def main():
    json_folder = "frames"  # JSON dosyalarının olduğu klasör
    output_json = "all_frames.json"  # Birleştirilmiş JSON
    merge_json_files(json_folder, output_json)  # JSON dosyalarını birleştir

    sentence = recognize_speech()  # Kullanıcının söylediği cümleyi al
    if sentence:
        video_frames = get_video_frames(output_json, sentence)  # Frame'leri al
        create_video(video_frames, "output_video.mp4")  # Videoyu oluştur

if __name__ == "__main__":
    main()

Merged JSON saved at all_frames.json
Konuşmanızı bekliyorum...
Algılanan metin: Merhaba ben
Uyarı: 'merhaba' kelimesi için frame bulunamadı.
Uyarı: 'ben' kelimesi için frame bulunamadı.
No frames loaded. Check file paths and JSON structure.
No frames to create a video.


In [20]:
import cv2
import os
import json

# Videoların bulunduğu klasör
video_folder = "İşaretDiliVideo"
# Çıktı framelerinin kaydedileceği klasör
output_frames_folder = "frames"
# JSON dosyası adı
output_json = "all_frames.json"

# Çıkış klasörünü oluştur
os.makedirs(output_frames_folder, exist_ok=True)

# JSON yapısını tutacak liste
all_frames = []

# Klasördeki tüm videoları işle
for video_file in os.listdir(video_folder):
    if video_file.endswith(".mp4"):  # Sadece mp4 dosyalarını al
        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]  # Video ismini al

        cap = cv2.VideoCapture(video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))  # FPS değerini al
        frame_id = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Frame dosya adını oluştur
            frame_filename = f"{video_name}_frame_{frame_id:04d}.jpg"
            frame_path = os.path.join(output_frames_folder, frame_filename)

            # Frame'i kaydet
            cv2.imwrite(frame_path, frame)

            # JSON'a eklenmek üzere frame bilgisi
            all_frames.append({
                "frame_id": frame_id,
                "video_name": video_name,
                "image_path": frame_path.replace("\\", "/"),  # Windows uyumu için
                "timestamp": frame_id / fps,  # Zaman damgası (saniye cinsinden)
                "keypoints": []  # İstersen buraya pose keypoints ekleyebilirsin
            })

            frame_id += 1

        cap.release()

# Tüm frameleri JSON dosyasına yaz
with open(output_json, "w") as json_file:
    json.dump({"frames": all_frames}, json_file, indent=4)

print(f"Tüm videolar framelendi ve JSON olarak kaydedildi: {output_json}")

Tüm videolar framelendi ve JSON olarak kaydedildi: all_frames.json
